### A python class for Graphs 

Let's begin with a adjacency matrix representation. We would like this detail to be hidden away and the user of this class should be able to use it transparently without concern about the underlying representation.

As we have seen in our reachability and connected components algorithms, we will need the ability to 
       * iterate over vertices
       * iterate over neighbours of a vertex.
       
We consider the iteration over vertices first and then iteration over neighbours.


In [ ]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjMat = np.zeros([self.N,self.N],dtype=np.int32)
        for e in E:
            x,y = e
            self.AdjMat[x,y]=1
            self.AdjMat[y,x]=1
            
    def __str__(self):
        return str(self.AdjMat)
        
    def __iter__(self):
        self.index = 0
        return(self)
        
    def __next__(self):
        if self.index >= self.N:
            raise StopIteration
        rval = self.index
        self.index = self.index+1
        return rval
    
            

In [ ]:
G = Graph(6,[(1,2),(2,3),(3,4),(4,1),(5,0)])

In [ ]:
H = Graph(3,[(0,1)])

In [ ]:
for u in G:
    for v in H:
        print(u,v)

In [ ]:
for u in G:
    for v in G:
        print(u,v)

This problem can be fixed by ensuring each iterator has its own state. 

So, we create a new object with the iterator so that it an keep this state.

In [ ]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjMat = np.zeros([self.N,self.N],dtype=np.int32)
        for e in E:
            x,y = e
            self.AdjMat[x,y]=1
            self.AdjMat[y,x]=1
            
    def __str__(self):
        return str(self.AdjMat)
    
    def __iter__(self):
        return self.Vertices(self.N)
    
    class Vertices:
        
        def __init__(self,nu):
            self.Nu = nu
            self.index = 0
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.index >= self.Nu:
                raise StopIteration
            rval = self.index
            self.index = self.index+1
            return rval
            

In [ ]:
G = Graph(6,[(1,2),(2,3),(3,4),(4,1),(5,0)])

In [ ]:
for i in G:
    for j in G:
        print(i,j)

We can go ahead and also get iterators for each vertex that allows us to iterate through their neighbours

In [1]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjMat = np.zeros([self.N,self.N],dtype=np.int32)
        for e in E:
            x,y = e
            self.AdjMat[x,y]=1
            self.AdjMat[y,x]=1
            
    def __str__(self):
        return str(self.AdjMat)
    
    def __iter__(self):
        return self.Vertices(self.N)
    
    def neighbours(self,v):
        return(self.Neighbours(self.AdjMat[v,],self.N))
    
    class Vertices:
        
        def __init__(self,nu):
            self.Nu = nu
            self.index = 0
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.index >= self.Nu:
                raise StopIteration
            rval = self.index
            self.index = self.index+1
            return rval
    
    class Neighbours:
        
        def __init__(self,row,nu):
            self.pos = 0
            self.row = row
            self.Nu = nu
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.pos >= self.Nu:
                raise StopIteration
            while self.row[self.pos] == 0: #continue till you get 1 or reach end of row
                self.pos = self.pos+1
                if self.pos >= self.Nu:
                    raise StopIteration
            rval = self.pos
            self.pos = self.pos+1
            return(rval)
            
            
            

In [ ]:
G = Graph(6,[(1,2),(2,3),(3,4),(4,1),(5,0)])

In [ ]:
for u in G:
    print(u," : ",end="")
    for v in G.neighbours(u):
        print(" ",v,end="")
    print()

In [ ]:
def reach(G,v):  # returns the set of vertices reachable from v
    UnExplored = {v}
    Explored = set()
    while UnExplored:
        u = UnExplored.pop()
        for v in G.neighbours(u):
            if v not in Explored and v not in UnExplored:
                UnExplored.add(v)
        Explored.add(u)
    return(Explored)

In [ ]:
reach(G,3)

In [ ]:
def breadthFS(G,v):
    UnExplored = [v]  # A list to maintain a queue.
    Explored = set()
    while UnExplored:
        u = UnExplored.pop(0)        # dequeue
        for v in G.neighbours(u):
            if v not in Explored and v not in UnExplored:
                UnExplored.append(v) # enqueue
        Explored.add(u)
    return(Explored)

#### breadthFS(G,3)

In [ ]:
def depthFS(G,v):
    UnExplored = [v]  # A list to maintain a stack.
    Explored = set()
    while UnExplored:
        u = UnExplored.pop()        # pop from stack
        for v in G.neighbours(u):
            if v not in Explored and v not in UnExplored:
                UnExplored.append(v) # push onto stack
        Explored.add(u)
    return(Explored)


In [ ]:
def depthFS(G,v): # Almost the same as above, except the neighbours are processed in reverse order.
    
    def dfs(u):
        Marked.add(u)
        for w in G.neighbours(u):
            if w not in Marked:
                dfs(w)
                
    Marked = set()
    dfs(v)
    return(Marked)
    

In [ ]:
depthFS(G,1)

In [ ]:
def ConnectedComponents(G,explore):
    visited = set()
    Components = []
    for v in G:
        if v not in visited:
            Components.append(explore(G,v))
        visited = visited | Components[-1]
    return Components

In [ ]:
ConnectedComponents(G,reach)

In [ ]:
ConnectedComponents(G,depthFS)

In [ ]:
ConnectedComponents(G,breadthFS)

###  Switching to Adjacency Lists

In [ ]:
import numpy as np
class Graph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjList = []
        for i in range(n):
            self.AdjList.append([])
        for e in E:
            x,y = e
            self.AdjList[x].append(y)
            self.AdjList[y].append(x)
            
    def __str__(self):
        return str(self.AdjList)
    
    def neighbours(self,v):
        return(self.Neighbours(self.AdjList[v]))
 
    def __iter__(self):
        return self.Vertices(self.N)
    
    class Vertices:
        
        def __init__(self,nu):
            self.Nu = nu
            self.index = 0
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.index >= self.Nu:
                raise StopIteration
            rval = self.index
            self.index = self.index+1
            return rval
    
    class Neighbours:
        
        def __init__(self,ls):
            self.pos = 0
            self.list = ls
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.pos >= len(self.list):
                raise StopIteration
            rval = self.list[self.pos]
            self.pos = self.pos+1
            return(rval)
            
            
            

In [ ]:
G = Graph(6,[(1,2),(2,3),(3,4),(4,1),(5,0)])
print(G)

In [ ]:
depthFS(G,2)

In [ ]:
breadthFS(G,2)

In [ ]:
import numpy as np
class DGraph:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjList = []
        for i in range(n):
            self.AdjList.append([])
        for e in E:
            x,y = e
            self.AdjList[x].append(y)
            
    def __str__(self):
        return str(self.AdjList)
    
    def __iter__(self):
        return self.Vertices(self.N)
    
    def neighbours(self,v):
        return(self.Neighbours(self.AdjList[v]))
    
    class Vertices:
        
        def __init__(self,nu):
            self.Nu = nu
            self.index = 0
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.index >= self.Nu:
                raise StopIteration
            rval = self.index
            self.index = self.index+1
            return rval
    
    class Neighbours:
        
        def __init__(self,ls):
            self.pos = 0
            self.list = ls
        
        def __iter__(self):
            return(self)
        
        def __next__(self):
            if self.pos >= len(self.list):
                raise StopIteration
            rval = self.list[self.pos]
            self.pos = self.pos+1
            return(rval)
            
            
            

In [ ]:
G = DGraph(6,[(1,2),(2,3),(3,4),(4,2),(0,5)])

In [ ]:
for v in G:
    print(v," : ",reach(G,v))

In [ ]:
def breadthFSwithDistance(G,v):
    UnExplored = [v]  # A list to maintain a queue.
    d = {}
    d[v] = 0
    Explored = set()
    while UnExplored:
        u = UnExplored.pop(0)        # dequeue
        for w in G.neighbours(u):
            if w not in Explored and w not in UnExplored:
                UnExplored.append(w) # enqueue
                d[w] = d[u] + 1
        Explored.add(u)
    return(d)

In [ ]:
for v in G:
    print(v," : ",breadthFSwithDistance(G,v))

You could also implement graphs without iterators as follows:

In [ ]:
class GraphTrivial:
    
    def __init__(self,n,E=[]):
        self.N = n
        self.AdjList = []
        for i in range(n):
            self.AdjList.append([])
        for e in E:
            x,y = e
            self.AdjList[x].append(y)
            self.AdjList[y].append(x)
            
    def __str__(self):
        return str(self.AdjList)
    
    def vertices(self):
        return(list(range(n)))
    
    def neighbours(self,v):
        return(self.AdjList[v].copy())

In [ ]:
G = GraphTrivial(6,[(1,2),(2,3),(3,4),(4,2),(0,5)])

In [2]:
g2=Graph(4,[(1,2),(1,3)])
